<a href="https://colab.research.google.com/github/DanielWarfield1/MLWritingAndResearch/blob/main/ReActInLangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ReAct
This article is a simultanious tutorial and exploration of ReAct within LangChain

sorry, it's a bit messy. I decided to go from LangChain to making ReAct from scratch.

In [ ]:
!pip -q install langchain huggingface_hub openai google-search-results tiktoken wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.4/794.4 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This b

In [ ]:
#copying from google drive to local
from google.colab import drive
import os
drive.mount('/content/drive')

with open ("/content/drive/My Drive/Colab Notebooks/Credentials/OpenAI-danielDemoKey.txt", "r") as myfile:
    OPENAI_API_TOKEN = myfile.read()
    os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN
print('API Key Loaded!')

Mounted at /content/drive
API Key Loaded!


In [ ]:
from langchain import OpenAI
from langchain import Wikipedia
from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer

In [ ]:
#defining a docstore, and telling the docstore
docstore=DocstoreExplorer(Wikipedia())

In [ ]:
docstore.search('Dune')

"Could not find [Dune]. Similar: ['Dune', 'Dune (2021 film)', 'Dune (novel)', 'Dune (franchise)', 'List of Dune characters', 'Dune: Part Two', 'Dune Messiah', 'Dune (disambiguation)', 'Sandworm (Dune)', 'Dune (1984 film)']"

In [ ]:
docstore.search('Sand Dune')

'A dune is a landform composed of wind- or water-driven sand. It typically takes the form of a mound, ridge, or hill. An area with dunes is called a dune system or a dune complex.  A large dune complex is called a dune field, while broad, flat regions covered with wind-swept sand or dunes with little or no vegetation are called ergs or sand seas. Dunes occur in different shapes and sizes, but most kinds of dunes are longer on the stoss (upflow) side, where the sand is pushed up the dune, and have a shorter slip face in the lee side. The valley or trough between dunes is called a dune slack.Dunes are most common in desert environments, where the lack of moisture hinders the growth of vegetation that would otherwise interfere with the  development of dunes. However, sand deposits are not restricted to deserts, and dunes are also found along sea shores, along streams in semiarid climates, in areas of glacial outwash, and in other areas where poorly cemented sandstone bedrock disintegrates

In [ ]:
docstore.search('Dune (novel)')

'Dune is a 1965 epic science fiction novel book by American author Frank Herbert, originally published as two separate serials (1963-64 novel \'Dune World\' and 1965 novel \'Prophet of Dune\') in Analog magazine. It tied with Roger Zelazny\'s This Immortal for the Hugo Award for Best Novel and won the inaugural Nebula Award for Best Novel in 1966. It is the first installment of the Dune Chronicles. It is one of the world\'s best-selling science fiction novels.Dune is set in the distant future in a feudal interstellar society in which various noble houses control planetary fiefs.  It tells the story of young Paul Atreides, whose family accepts the stewardship of the planet Arrakis. While the planet is an inhospitable and sparsely populated desert wasteland, it is the only source of melange, or "spice", a drug that extends life and enhances mental abilities. Melange is also necessary for space navigation, which requires a kind of multidimensional awareness and foresight that only the dru

In [ ]:
docstore.lookup('Lawrence')

'(Result 1/2) \n== Origins ==\nAfter his novel The Dragon in the Sea was published in 1957, Herbert traveled to Florence, Oregon, at the north end of the Oregon Dunes. Here, the United States Department of Agriculture was attempting to use poverty grasses to stabilize the sand dunes. Herbert claimed in a letter to his literary agent, Lurton Blassingame, that the moving dunes could "swallow whole cities, lakes, rivers, highways." Herbert\'s article on the dunes, "They Stopped the Moving Sands", was never completed (and only published decades later in The Road to Dune), but its research sparked Herbert\'s interest in ecology and deserts.Herbert further drew inspiration from Native American mentors like "Indian Henry" (as Herbert referred to the man to his son; likely a Henry Martin of the Hoh tribe) and Howard Hansen. Both Martin and Hansen grew up on the Quileute reservation near Herbert\'s hometown. According to historian Daniel Immerwahr, Hansen regularly shared his writing with Herbe

In [ ]:
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

# llm = OpenAI(temperature=0, model_name="text-davinci-003")
llm = OpenAI(temperature=0, model_name="gpt-3.5-turbo-instruct")
react_agent = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

In [ ]:
print(react_agent.agent.llm_chain.prompt.template)



Question: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?
Thought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.
Action: Search[Colorado orogeny]
Observation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.
Thought: It does not mention the eastern sector. So I need to look up eastern sector.
Action: Lookup[eastern sector]
Observation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.
Thought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.
Action: Search[High Plains]
Observation: High Plains refers to one of two distinct land regions
Thought: I need to instead search High Plains (United States).
Action: Search[High Plains (United St

In [ ]:
react_agent.agent.llm_chain

LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='\n\nQuestion: What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?\nThought: I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.\nAction: Search[Colorado orogeny]\nObservation: The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.\nThought: It does not mention the eastern sector. So I need to look up eastern sector.\nAction: Lookup[eastern sector]\nObservation: (Result 1 / 1) The eastern sector extends into the High Plains and is called the Central Plains orogeny.\nThought: The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.\nAction: Search[High Plains]\nObservation: High Plains refers to one of two distinct land regions\n

In [ ]:
prompt = "What is the age of the president of the United States? The current date is Dec 26 2023."
react_agent.run(prompt)



> Entering new AgentExecutor chain...
Thought: I need to find the current president of the United States and their age on December 26, 2023.
Action: Search[current president of the United States]
Observation: The president of the United States (POTUS) is the head of state and head of government of the United States of America. The president directs the executive branch of the federal government and is the commander-in-chief of the United States Armed Forces.
The power of the presidency has grown substantially since the first president, George Washington, took office in 1789. While presidential power has ebbed and flowed over time, the presidency has played an increasingly significant role in American political life since the beginning of the 20th century, carrying over into the 21st century with notable expansions during the presidencies of Franklin D. Roosevelt and George W. Bush. In modern times, the president is one of the world's most powerful political figures and the leader of 

'81 years old'

In [ ]:
from langchain.chat_models import ChatOpenAI

prompt = "Calculate the cumulative age of the oldest cities in all the regions in Italy. You have access to wikipedia given the 'search' and 'lookup' tools."
llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview")
react_agent = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)
react_agent.run(prompt)



> Entering new AgentExecutor chain...
To calculate the cumulative age of the oldest cities in all the regions in Italy, I would need to identify each region in Italy, find the oldest city in each region, and then sum up their ages. Since this is a complex task that requires multiple searches and calculations, I will outline the steps I would take to complete it:

1. Identify all the regions in Italy.
2. For each region, find the oldest city.
3. Determine the founding year of each oldest city.
4. Calculate the age of each city by subtracting the founding year from the current year (2023).
5. Sum up the ages of all the oldest cities to get the cumulative age.

Let's start with the first step:

Action: Search[Regions of Italy]
Observation: The regions of Italy (Italian: regioni d'Italia) are the first-level administrative divisions of the Italian Republic, constituting its second NUTS administrative level. There are twenty regions, five of which have higher autonomy than the rest. Under

ValueError: ignored

In [ ]:
import wikipedia

In [ ]:
wikipedia.search("Dune")

['Dune',
 'Dune (2021 film)',
 'Dune (novel)',
 'Dune (franchise)',
 'List of Dune characters',
 'Dune: Part Two',
 'Dune Messiah',
 'Dune (disambiguation)',
 'Dune (1984 film)',
 'Sandworm (Dune)']

In [ ]:
pg = wikipedia.page("Dune")

PageError: ignored

In [ ]:
pg = wikipedia.page("Dune_")

In [ ]:
pg = wikipedia.page("Dune (novel)")